In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from pandas.io.json import json_normalize

mongoConn = MongoClient(host='127.0.0.1', port=27017)  # 实例化MongoClient
db = mongoConn.get_database('stock')  # 连接数据库

positions = db.get_collection('historialstock') # 连接集合
mon_data = positions.find({'code':{'$in':['GOOG','AMZN','FB','AAPL','BABA']}}) 
# 查询这个集合下的所有记录,数据以json格式返回
data = json_normalize([comment for comment in mon_data])
data.head()

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,_id,date,code,name,closingPrice,openingPrice,maxPrice,minPrice,volume
0,5e6b8033655221b99c0af14f,2020-03-11,AAPL,苹果,275.43,277.39,281.22,271.86,64094970
1,5e6b8033655221b99c0af150,2020-03-10,AAPL,苹果,285.34,277.14,286.44,269.37,71322520
2,5e6b8033655221b99c0af151,2020-03-09,AAPL,苹果,266.17,263.75,278.09,263,71686210
3,5e6b8033655221b99c0af152,2020-03-06,AAPL,苹果,289.03,282,290.82,281.23,56544250
4,5e6b8033655221b99c0af153,2020-03-05,AAPL,苹果,292.92,295.52,299.55,291.41,46893220


In [2]:
data[data['code']=='BABA']

,_id,date,code,name,closingPrice,openingPrice,maxPrice,minPrice,volume
5034,5e6b807c655221b99c18ea8b,2020-03-11,BABA,阿里巴巴,198.91,201.65,203.1,196.11,18931800
5035,5e6b807c655221b99c18ea8c,2020-03-10,BABA,阿里巴巴,206.39,205.53,207.45,200.8,19022620
5036,5e6b807c655221b99c18ea8d,2020-03-09,BABA,阿里巴巴,197.66,195.62,199.89,193.93,25502570
5037,5e6b807c655221b99c18ea8e,2020-03-06,BABA,阿里巴巴,204.64,206.7,207,201.1,21700510
5038,5e6b807c655221b99c18ea8f,2020-03-05,BABA,阿里巴巴,211.46,210,215.15,209.14,13462890
...,...,...,...,...,...,...,...,...,...
6407,5e6b807c655221b99c18efe8,2014-09-25,BABA,阿里巴巴,88.92,91.09,91.5,88.5,28566180
6408,5e6b807c655221b99c18efe9,2014-09-24,BABA,阿里巴巴,90.57,88.47,90.57,87.22,31966400
6409,5e6b807c655221b99c18efea,2014-09-23,BABA,阿里巴巴,87.17,88.94,90.48,86.62,38935050
6410,5e6b807c655221b99c18efeb,2014-09-22,BABA,阿里巴巴,89.89,92.7,92.95,89.5,66495310


In [3]:
def change(closingPrice,openingPrice):
    cp = float(closingPrice)
    op = float(openingPrice)
    
    return ((cp-op)/op)*100

data['change'] = data.apply(lambda x:change(x['closingPrice'],x['openingPrice']), axis=1)
data['closingPrice'] = data['closingPrice'].astype("float")
data['openingPrice'] = data['openingPrice'].astype("float")
data['maxPrice'] = data['maxPrice'].astype("float")
data['minPrice'] = data['minPrice'].astype("float")
data['volume'] = data['volume'].astype("float")
#data[data['code']=='AAPL'].isnull().sum()
data[data['code']=='AAPL'].describe()

,closingPrice,openingPrice,maxPrice,minPrice,volume,change
count,2517.000000,2517.000000,2517.000000,2517.000000,2.517000e+03,2517.000000
mean,115.618176,115.569327,116.632991,114.513751,7.231386e+07,0.016461
std,61.243458,61.105091,61.747873,60.610900,5.641731e+07,1.286398
min,31.750000,31.495700,32.140000,28.464300,1.136205e+07,-6.633066
25%,68.107100,68.178600,68.911400,67.100000,3.052172e+07,-0.705992
50%,101.630000,101.590000,102.380000,100.860000,5.290232e+07,0.046607
75%,155.450000,155.480000,156.730000,154.000000,9.787438e+07,0.760456
max,327.200000,324.740000,327.850000,323.350000,4.624423e+08,8.696110


In [47]:
from pyecharts import Line
line =Line("股价走势")
line.add('FB', 
         data[data['code']=='FB']['date'][::-1], 
         data[data['code']=='FB']['closingPrice'][::-1], is_label_show=True) 
line.add('AAPL', 
         data[data['code']=='AAPL']['date'][::-1], 
         data[data['code']=='AAPL']['closingPrice'][::-1], is_label_show=True) 
line.add('BABA', 
         data[data['code']=='BABA']['date'][::-1], 
         data[data['code']=='BABA']['closingPrice'][::-1], is_label_show=True) 
line.add('GOOG', 
         data[data['code']=='GOOG']['date'][::-1], 
         data[data['code']=='GOOG']['closingPrice'][::-1], is_label_show=True) 
line.add('AMZN', 
         data[data['code']=='AMZN']['date'][::-1], 
         data[data['code']=='AMZN']['closingPrice'][::-1], is_label_show=True) 

#line.render()


In [19]:
from pyecharts import Line

data['date1'] = pd.to_datetime(data['date'])
_data = data[data['date1']>'2018-12-31']

line = Line("股价走势")

line.add('GOOG', 
         _data[_data['code']=='GOOG']['date'][::-1], 
         _data[_data['code']=='GOOG']['closingPrice'][::-1], is_label_show=True) 
line.add('AMZN', 
         _data[_data['code']=='AMZN']['date'][::-1], 
         _data[_data['code']=='AMZN']['closingPrice'][::-1], is_label_show=True,yaxis_line_width=4,xaxis_line_width=4,yaxis_label_textsize=16,xaxis_label_textsize=16) 
#line.render()

In [18]:
line1 = Line("股价走势")
line1.add('FB', 
         _data[_data['code']=='FB']['date'][::-1], 
         _data[_data['code']=='FB']['closingPrice'][::-1], is_label_show=True) 
line1.add('AAPL', 
         _data[_data['code']=='AAPL']['date'][::-1], 
         _data[_data['code']=='AAPL']['closingPrice'][::-1], is_label_show=True) 
line1.add('BABA', 
         _data[_data['code']=='BABA']['date'][::-1], 
         _data[_data['code']=='BABA']['closingPrice'][::-1], is_label_show=True,yaxis_line_width=4,xaxis_line_width=4,yaxis_label_textsize=16,xaxis_label_textsize=16) 
#line1.render()

In [17]:
line2 =Line("股价走势")
line2.add('GOOG', 
         _data[_data['code']=='GOOG']['date'][::-1], 
         _data[_data['code']=='GOOG']['closingPrice'][::-1], is_label_show=True) 
line2.add('AMZN', 
         _data[_data['code']=='AMZN']['date'][::-1], 
         _data[_data['code']=='AMZN']['closingPrice'][::-1], is_label_show=True) 
line2.add('FB', 
         _data[_data['code']=='FB']['date'][::-1], 
         _data[_data['code']=='FB']['closingPrice'][::-1], is_label_show=True) 
line2.add('AAPL', 
         _data[_data['code']=='AAPL']['date'][::-1], 
         _data[_data['code']=='AAPL']['closingPrice'][::-1], is_label_show=True) 
line2.add('BABA', 
         _data[_data['code']=='BABA']['date'][::-1], 
         _data[_data['code']=='BABA']['closingPrice'][::-1], is_label_show=True,yaxis_line_width=4,xaxis_line_width=4,yaxis_label_textsize=16,xaxis_label_textsize=16) 

In [18]:
def total_change(code):
    cur = data[data['code']==code].head(1)['closingPrice'].values
    before = data[data['code']==code].tail(1)['closingPrice'].values
    
    change = ((cur-before)/before)*100
    print('该股票累计增长:'+str(change[0])+'%')
    


In [19]:
total_change('GOOG')

该股票累计增长:117.63599899724242%


In [20]:
total_change('AMZN')

该股票累计增长:1281.3230162342588%


In [21]:
total_change('FB')

该股票累计增长:345.2837690090449%


In [22]:
total_change('AAPL')

该股票累计增长:750.8436459343741%


In [23]:
total_change('BABA')

该股票累计增长:111.85429758227711%


In [16]:
from pyecharts import Bar
bar =Bar("条形图", "平均股价比较")
x = ['GOOG','AMZN','FB','AAPL','BABA']
y = [data[data['code']=='GOOG']['closingPrice'].mean(),
     data[data['code']=='AMZN']['closingPrice'].mean(),
    data[data['code']=='FB']['closingPrice'].mean(),
    data[data['code']=='AAPL']['closingPrice'].mean(),
    data[data['code']=='BABA']['closingPrice'].mean()]
bar.add("平均股价",x,y,yaxis_line_width=4,xaxis_line_width=4,yaxis_label_textsize=16,xaxis_label_textsize=16)